In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import numpy as np

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
def build_seir_model(
    config: dict,
) -> CompartmentalModel:

    # Model characteristics
    compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
    model = CompartmentalModel(
        times=(0., config["end_time"]),
        compartments=compartments,
        infectious_compartments=["Infectious"],
        timestep=config["t_step"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": config["tot_popn"] - config["infectious_seed"],
            "Infectious": config["infectious_seed"]
        }
    )
    
    # Transitions
    ave_infous = Parameter("ave_infous")
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("r0") / ave_infous,
        source="Susceptible",
        dest="Pre-infectious"
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=1. / Parameter("ave_preinfous"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / ave_infous,
        source="Infectious", 
        dest="Immune"
    )
    
    # Outputs
    model.request_output_for_flow(
        name="incidence", 
        flow_name="progression"
    )
    model.request_output_for_compartments(
        name="total_population",
        compartments=compartments,
        save_results=False,
    )
    model.request_output_for_compartments(
        name="susceptible",
        compartments=["Susceptible"],
        save_results=False,
    )
    model.request_output_for_compartments(
        name="immune",
        compartments=["Immune"],
        save_results=False,
    )
    model.request_function_output(
        name="susc_prop",
        func=DerivedOutput("susceptible") / DerivedOutput("total_population"),
    )
    model.request_function_output(
        name="immue_prop",
        func=DerivedOutput("immune") / DerivedOutput("total_population"),
    )
    model.request_function_output(
        name="r_n",
        func=DerivedOutput("susc_prop") * Parameter("r0"),
    )
    
    return model

In [ ]:
model_config = {
    "tot_popn": 1e5,
    "infectious_seed": 1.,
    "end_time": 150.,
    "t_step": 1.,
}

parameters = {
    "r0": 2.,
    "ave_preinfous": 2.,
    "ave_infous": 2.,
    "life_expectancy": 70.,
}

In [ ]:
seir_model = build_seir_model(model_config)
seir_model.run(parameters=parameters)
derived_outputs = seir_model.get_derived_outputs_df()

#### Figure 2
Figure 1 only contains information in Figure 2.

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(seir_model.times, derived_outputs["susc_prop"], color="navy", label="Susceptible")
left_ax.plot(seir_model.times, derived_outputs["immue_prop"], color="lime", linestyle="--", label="Immune")
left_ax.set_xlim(0, 120.)
left_ax.set_xlabel("Time (days)")
left_ax.set_ylim(0, 1.)
left_ax.set_ylabel("Proportion of susceptible or immune individuals")
left_ax.xaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.legend()
left_ax.spines.top.set_visible(False)

twin_ax = left_ax.twinx()
twin_ax.plot(seir_model.times, derived_outputs["incidence"], color="magenta", linestyle="--", label="New infectious persons")
twin_ax.set_ylim(0, 8e3)
twin_ax.set_ylabel("Number of new infectious persons/time step")
twin_ax.legend(loc=5)
twin_ax.spines.top.set_visible(False)

image = plt.imread("./figures/model_4_1a_fig_1.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(seir_model.times, derived_outputs["susc_prop"], color="navy", label="Susceptible")
left_ax.plot(seir_model.times, derived_outputs["immue_prop"], color="lime", linestyle="--", label="Immune")
left_ax.plot(seir_model.times, derived_outputs["r_n"], color="black", label="Rn")

left_ax.set_xlim(0, 120.)
left_ax.set_xlabel("Time (days)")
left_ax.set_ylim(0, 2)
left_ax.set_yticks(np.linspace(0., 2., 5))
left_ax.set_ylabel("Proportion of susceptible or immune individuals and Rn")
left_ax.xaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.legend()
left_ax.set_title("Figure 2")
left_ax.spines.top.set_visible(False)

twin_ax = left_ax.twinx()
twin_ax.plot(seir_model.times, derived_outputs["incidence"], color="magenta", linestyle="--", label="New infectious persons")
twin_ax.set_ylim(0, 8e3)
twin_ax.set_ylabel("Number of new infectious persons/time step")
twin_ax.legend(loc=5)
twin_ax.spines.top.set_visible(False)

image = plt.imread("./figures/model_4_1a_fig_2.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");

#### Figure 3

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(seir_model.times, derived_outputs["incidence"], color="magenta", linestyle="--")
left_ax.set_xlim(0, 2e3)
left_ax.set_xlabel("Time (days)")
left_ax.set_xticks(range(0, 2500, 500))
left_ax.xaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.set_yscale("log")
left_ax.set_ylim(1e-2, 1e4)
left_ax.set_ylabel("Number of new infectious persons per time step")
left_ax.set_title("Figure 3")
left_ax.spines.top.set_visible(False)
left_ax.spines.right.set_visible(False)

image = plt.imread("./figures/model_4_1a_fig_3.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");